In [1]:
import os
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.api.types import EmbeddingFunction
from sklearn.model_selection import train_test_split
import requests
import time


c:\Users\firas\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
filename_data = '../datacreation/dialogues_embededd.pkl'
data = pd.read_pickle(filename_data)
train_data, test_data = train_test_split(data, test_size=0.05)
print("Train shape:", train_data.shape)
print("Test shape:", test_data.shape)


Train shape: (950, 6)
Test shape: (50, 6)


In [3]:
df = pd.read_csv('../datacreation/dialogues.csv', sep='\t')
df = df.dropna()
df.rename(columns={'Description':'Question', 'Doctor':'Answer'}, inplace=True)
df['combined'] = 'Question: ' + df.Question.str.strip() + '\nAnswer: ' + df.Answer.str.strip()
df = df.drop_duplicates().reset_index(drop=True)
df['ids'] = df.index
documents = df
print(documents.head(2))


                                            Question  \
0        Will masturbation cause weakness in nerves?   
1  Could lack of hair growth be due to masturbation?   

                                             Patient  \
0  Hi, may I answer your health queries right now...   
1  hai sir i am 25years old i used to do masturba...   

                                              Answer  \
0  Hi, Masturbation does make the nerves weak whe...   
1  Hi, Since you said you masturbate frequently e...   

                                            combined  ids  
0  Question: Will masturbation cause weakness in ...    0  
1  Question: Could lack of hair growth be due to ...    1  


In [4]:
class MiniLML6V2EmbeddingFunction(EmbeddingFunction):
    MODEL = SentenceTransformer('all-MiniLM-L6-v2')
    def __call__(self, texts):
        return MiniLML6V2EmbeddingFunction.MODEL.encode(texts).tolist()
emb_func = MiniLML6V2EmbeddingFunction()

C:\Users\firas\AppData\Local\Temp\ipykernel_3768\1187675410.py:5: DeprecationWarning: The class MiniLML6V2EmbeddingFunction does not implement __init__. This will be required in a future version.
  emb_func = MiniLML6V2EmbeddingFunction()


In [8]:
class ChromaWithUpsert:
    def __init__(
            self,
            name='openai_rag_collection',
            persist_directory=None,
            embedding_function=None,
            collection_metadata=None,
    ):
        if persist_directory is not None:
            self._client = chromadb.PersistentClient(path=persist_directory)
        else:
            self._client = chromadb.EphemeralClient()
        self._embedding_function = embedding_function
        self._persist_directory = persist_directory
        self._name = name
        self._collection = self._client.get_or_create_collection(
            name=self._name,
            embedding_function=self._embedding_function
            if self._embedding_function is not None
            else None,
            metadata=collection_metadata,
        )
    def upsert_texts(
        self,
        texts,
        metadata=None,
        ids=None,
        **kwargs
    ):
        texts_list = list(texts)
        if ids is None:
            import uuid
            ids = [str(uuid.uuid1()) for _ in texts_list]
        batch_size = 5000
        for i in range(0, len(texts_list), batch_size):
            end = min(i + batch_size, len(texts_list))
            batch_texts = texts_list[i:end]
            batch_ids = ids[i:end]
            batch_metadata = None if metadata is None else metadata[i:end]
            self._collection.upsert(
                metadatas=batch_metadata, 
                documents=batch_texts, 
                ids=batch_ids
            )
        return ids

    def is_empty(self):
        return self._collection.count() == 0

    def persist(self):
        if hasattr(self._client, 'persist'):
            self._client.persist()

    def query(self, query_texts, n_results=5):
        return self._collection.query(query_texts=query_texts, n_results=n_results)



In [9]:
knowledge_base_dir = '../datacreation/knowledge'  # or your actual path
chroma = ChromaWithUpsert(
    name='openai_rag_collection',
    embedding_function=emb_func,
    persist_directory=knowledge_base_dir,
)
print("Knowledge base loaded.")

Knowledge base loaded.


In [10]:
question_texts = [q.strip("?") + "?" for q in test_data['Question'].tolist()]
print("\n".join(question_texts[:5]))

What are the white dots around my scrotum?
What does this Semen analysis for fertility indicate?
What causes penis to become small and semen discharge?
What is the treatment for brown color semen discharge?
What causes sperm leakage during urination?


In [11]:
relevant_contexts = []
for question_text in question_texts:
    relevant_chunks = chroma.query(
        query_texts=[question_text],
        n_results=5,
    )
    relevant_contexts.append(relevant_chunks)


In [12]:
def make_prompt(context, question_text):
    return (f"Veuillez répondre à la question suivante en vous appuyant sur le contexte fourni.\n"
            f"{context}\n\n"
            f"Question : {question_text}")

prompt_texts = []
for relevant_context, question_text in zip(relevant_contexts, question_texts):
    context = "\n\n".join(relevant_context["documents"][0])
    prompt_text = make_prompt(context, question_text)
    prompt_texts.append(prompt_text)

print(prompt_texts[0])

Veuillez répondre à la question suivante en vous appuyant sur le contexte fourni.


Question : What are the white dots around my scrotum?


In [14]:
def generate_ollama_answer(prompt, model="llama3"):
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": model,
            "prompt": prompt,
            "stream": False
        }
    )
    response.raise_for_status()
    return response.json()["response"]

results = []
for prompt_text in prompt_texts[:10]:  # Limit to 10 for testing
    try:
        answer = generate_ollama_answer(prompt_text)
        results.append(answer)
        time.sleep(1)  # To avoid overloading Ollama
    except Exception as e:
        print("Erreur Ollama:", e)
        results.append("Erreur API")

# 11. Display results
for q, a in zip(question_texts[:10], results):
    print("Q:", q)
    print("A:", a)
    print("="*60)

Q: What are the white dots around my scrotum?
A: A very specific and curious question!

The white dots you're referring to are likely Milia. Milia are small, benign bumps that can appear on the skin of the genital area, including the scrotum. They are usually small, whitish or yellowish in color, and may be flat or raised.

Milia are caused by a buildup of keratin, a protein found in skin cells, within tiny pockets of skin. This buildup creates small cysts that can appear as white dots. Milia are harmless and do not require treatment unless they become inflamed or irritated.

Other possibilities for white dots around the scrotum could be:

* Seborrheic keratoses: benign growths caused by sun exposure
* Eczema or dermatitis: skin irritation or inflammation that can cause small, whitish bumps
* Inflammatory lesions: small red or white bumps caused by infections or allergic reactions

It's always a good idea to consult with a healthcare professional if you're concerned about any unusual s